### Step 1: Install Required Packages


In [ ]:
# Install required packages
!pip install -q fastapi uvicorn pyngrok ultralytics
!pip install -q nest-asyncio

### Step 2: Mount Google Drive


In [ ]:
from google.colab import drive

drive.mount("/content/drive")

### Step 3: Setup VS Code SSH Tunnel (Optional)


In [ ]:
# Optional: Setup SSH tunnel for VS Code Remote
# !pip install colab_ssh --upgrade
# from colab_ssh import launch_ssh_cloudflared
# launch_ssh_cloudflared(password="your_password_here")

### Step 4: Create FastAPI Training Server


In [ ]:
from fastapi import FastAPI, BackgroundTasks, HTTPException
from pydantic import BaseModel
from typing import Optional, Dict
import uvicorn
import nest_asyncio
from datetime import datetime
import os
import logging

# Allow nested event loops
nest_asyncio.apply()

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Create FastAPI app
app = FastAPI(title="Colab Training Server")

# Global training status
training_status = {
    "status": "idle",
    "progress": 0,
    "message": "Ready to train",
    "started_at": None,
    "completed_at": None,
    "model_path": None,
}


class TrainingRequest(BaseModel):
    epochs: int = 10
    batch_size: int = 16
    model: str = "yolov8n.pt"
    data_path: str = "/content/drive/MyDrive/harmful_detection_dataset"


@app.get("/")
async def root():
    return {
        "message": "Colab Training Server is running",
        "status": training_status["status"],
    }


@app.get("/status")
async def get_status():
    return training_status


@app.post("/train")
async def start_training(request: TrainingRequest, background_tasks: BackgroundTasks):
    global training_status

    if training_status["status"] == "training":
        raise HTTPException(status_code=400, detail="Training already in progress")

    # Update status
    training_status["status"] = "training"
    training_status["progress"] = 0
    training_status["message"] = "Starting training..."
    training_status["started_at"] = datetime.now().isoformat()

    # Start training in background
    background_tasks.add_task(train_model, request)

    return {"message": "Training started", "request": request.dict()}


def train_model(request: TrainingRequest):
    """
    Train YOLO model
    """
    global training_status

    try:
        from ultralytics import YOLO

        logger.info(f"Starting training with {request.epochs} epochs")

        # Load model
        model = YOLO(request.model)

        # Check if data.yaml exists
        data_yaml = os.path.join(request.data_path, "data.yaml")

        if not os.path.exists(data_yaml):
            training_status["status"] = "failed"
            training_status["message"] = f"data.yaml not found at {data_yaml}"
            logger.error(training_status["message"])
            return

        training_status["progress"] = 10
        training_status["message"] = "Model loaded, starting training..."

        # Train model
        results = model.train(
            data=data_yaml,
            epochs=request.epochs,
            batch=request.batch_size,
            imgsz=640,
            device=0,  # Use GPU
            project="/content/drive/MyDrive/harmful_detection_models",
            name=f"train_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        )

        training_status["progress"] = 90
        training_status["message"] = "Training completed, saving model..."

        # Get best model path
        best_model = results.save_dir / "weights" / "best.pt"

        training_status["status"] = "completed"
        training_status["progress"] = 100
        training_status["message"] = "Training completed successfully"
        training_status["completed_at"] = datetime.now().isoformat()
        training_status["model_path"] = str(best_model)

        logger.info(f"Training completed. Model saved at: {best_model}")

    except Exception as e:
        training_status["status"] = "failed"
        training_status["message"] = f"Training failed: {str(e)}"
        logger.error(f"Training failed: {e}")


@app.post("/reset")
async def reset_status():
    global training_status
    training_status = {
        "status": "idle",
        "progress": 0,
        "message": "Ready to train",
        "started_at": None,
        "completed_at": None,
        "model_path": None,
    }
    return {"message": "Status reset"}


print("FastAPI server configured")

### Step 5: Setup ngrok Tunnel (Alternative to VS Code)


In [ ]:
# Optional: Use ngrok to expose the API
from pyngrok import ngrok

# Set your ngrok auth token (get it from https://ngrok.com)
# ngrok.set_auth_token("YOUR_NGROK_TOKEN")

# Create tunnel
public_url = ngrok.connect(8000)
print(f"\n🌐 Public URL: {public_url}")
print(f"\n📝 Use this URL in Airflow HTTP connection: {public_url}")

### Step 6: Start the Server


In [ ]:
# Start the FastAPI server
print("🚀 Starting FastAPI server on port 8000...")
print("\n📡 Server will be accessible at:")
print("   - Local: http://localhost:8000")
print("   - Docs: http://localhost:8000/docs")
print("\n⚙️ Training endpoint: POST /train")
print("📊 Status endpoint: GET /status")
print("\n⏹️ Press Ctrl+C to stop the server\n")

uvicorn.run(app, host="0.0.0.0", port=8000)

### Test the API


In [ ]:
# Test the API endpoints
import requests

# Test root endpoint
response = requests.get("http://localhost:8000/")
print("Root:", response.json())

# Test status endpoint
response = requests.get("http://localhost:8000/status")
print("Status:", response.json())

# Test training endpoint (uncomment to actually start training)
# training_request = {
#     "epochs": 2,
#     "batch_size": 8,
#     "model": "yolov8n.pt",
#     "data_path": "/content/drive/MyDrive/harmful_detection_dataset"
# }
# response = requests.post("http://localhost:8000/train", json=training_request)
# print("Training:", response.json())

### Usage Instructions

1. **Run all cells** to start the FastAPI server
2. **Use ngrok** to get a public URL (or use VS Code tunnel)
3. **In Airflow UI**:
   - Go to Admin > Connections
   - Create new HTTP connection with ID: `colab_local_conn`
   - Set Host to your ngrok URL (or localhost:8000 if using VS Code tunnel)
4. **Trigger the DAG** in Airflow to start training
5. **Monitor progress** at `/status` endpoint
6. **Download trained model** from Google Drive after completion
